# Testing basic tools in documentation

In [ ]:
import molsysmt as msm

In [ ]:
molecular_system_A = msm.build.build_peptide('AceProNme')
molecular_system_B = msm.build.build_peptide('AceValNme')
molecular_system_C = msm.build.build_peptide('AceLysNme')

In [ ]:
molecular_system_B = msm.structure.translate(molecular_system_B, translation='[-1.0, 0.0, 0.0] nanometers')
molecular_system_C = msm.structure.translate(molecular_system_C, translation='[1.0, 0.0, 0.0] nanometers')

In [ ]:
msm.add(molecular_system_A, molecular_system_B, keep_ids=False)

In [ ]:
molecular_system_A.topology.chains